In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys
sys.path.append("../")
import numpy as np
import numpy.linalg as LA
from estimators.Tyler_estimators import TRex_random_init, Tyler_standard_FP
from experiments.utils import generate_samples, scatter_to_corr, build_true_cov, errors_corr

In [2]:
np.random.seed(0)
data = pd.read_csv("data/sp500_cleaned.csv")
returns = data.pct_change().dropna().values.T
n_tot = returns.shape[0]
m_tot = returns.shape[1]
num_runs = 30
all_n = [50, 100, 150, 200, 250, 300, 350, 400, 450]
num_factors = 5
errors_Tyler = np.zeros((num_runs, len(all_n)))
errors_TRex = np.zeros((num_runs, len(all_n)))
times_Tyler = np.zeros((num_runs, len(all_n)))
times_TRex = np.zeros((num_runs, len(all_n)))
MAX_ITER = 20

F = np.random.randn(30, num_factors)
d = np.random.rand(30)
F = np.ascontiguousarray(F)
d = np.ascontiguousarray(d)
X = returns[0:30, :]
X = X - np.mean(X, axis=1, keepdims=True)
X2 = np.ascontiguousarray(X)
F_TRex, d_TRex = \
TRex_random_init(X2, num_factors, F, d, 
                 outer_max_iter=20, inner_max_iter=100)


In [4]:
for ii in range(len(all_n)):
    print(f'simulating n = {all_n[ii]}... ')
    n = all_n[ii]
    cov_true = np.cov(returns[0:n, :])
    corr_true = scatter_to_corr([cov_true])[0]
    m = n + 1
    for run in range(num_runs):     
        X = generate_samples(m, "N", {'mu': np.zeros((n, )), 'cov': cov_true})
        X = X - np.mean(X, axis=1, keepdims=True)
        
        # estimation using vanilla Tyler
        start = time.time()
        scatter_Tyler = Tyler_standard_FP(X, MAX_ITER)
        end = time.time()
        times_Tyler[run, ii] = end - start
        
        # compute factor model via Tyler
        X2 = np.ascontiguousarray(X)
        F_init = np.random.randn(X2.shape[0], num_factors)
        d_init = np.random.rand(X2.shape[0])
        F_init = np.ascontiguousarray(F_init)
        d_init = np.ascontiguousarray(d_init) 
        start = time.time()
        F_TRex, d_TRex = \
            TRex_random_init(X2, num_factors, F_init, d_init,
                             outer_max_iter=MAX_ITER,
                             inner_max_iter=100)
        end = time.time()
        times_TRex[run, ii] = end - start 
                    
        scatter_TRex = F_TRex @ F_TRex.T + np.diag(d_TRex)        
        errors = errors_corr([scatter_Tyler, scatter_TRex], true_corr=corr_true)
        errors_Tyler[run, ii] = errors[0]
        errors_TRex[run, ii] = errors[1]        

        #print(f"Tyler time: {times_Tyler[run, ii]}, TRex time: {times_TRex[run, ii]}")
        #print(f"Tyler error: {errors[0]}, TRex error: {errors[1]}") 



simulating n = 50... 
simulating n = 100... 


KeyboardInterrupt: 

In [ ]:
average_Tyler_errors = np.mean(errors_Tyler, axis=0)
average_TRex_errors = np.mean(errors_TRex, axis=0)
average_Tyler_times = np.mean(times_Tyler, axis=0)
average_TRex_times = np.mean(times_TRex, axis=0)

std_Tyler_errors = np.std(errors_Tyler, axis=0)
std_TRex_errors = np.std(errors_TRex, axis=0)
std_Tyler_times = np.std(times_Tyler, axis=0)
std_TRex_times = np.std(times_TRex, axis=0)

fig, axs = plt.subplots(1, 2, figsize=(20, 8))
MARKERSIZE, CAPSIZE, CAPTHICK, ELINEWIDTH, LINEWIDTH = 12, 5, 4, 4, 4
FONTSIZE_Y, FONTSIZE_X = 30, 25

axs[1].errorbar(all_n, average_Tyler_errors, 
                yerr=std_Tyler_errors, 
                marker='o', linestyle='--',  linewidth=LINEWIDTH,
                markersize=30, capsize=CAPSIZE, capthick=CAPTHICK, 
                elinewidth=ELINEWIDTH, label="Tyler")

axs[1].errorbar(all_n, average_TRex_errors, 
                yerr=std_TRex_errors, 
                marker='v', linestyle='--',  linewidth=LINEWIDTH,
                markersize=25, capsize=CAPSIZE, capthick=CAPTHICK, 
                elinewidth=ELINEWIDTH, label="T-Rex (Ours)")


axs[0].errorbar(all_n, average_Tyler_times, 
                yerr=std_Tyler_times, 
                marker='o', linestyle='--',  linewidth=LINEWIDTH,
                markersize=30, capsize=CAPSIZE, capthick=CAPTHICK, 
                elinewidth=ELINEWIDTH)

axs[0].errorbar(all_n, average_TRex_times, 
                yerr=std_TRex_times, 
                marker='v', linestyle='--',  linewidth=LINEWIDTH,
                markersize=25, capsize=CAPSIZE, capthick=CAPTHICK, 
                elinewidth=ELINEWIDTH)



axs[0].grid()
axs[1].grid()
axs[0].tick_params(axis='both', which='major', labelsize=20)
axs[1].tick_params(axis='both', which='major', labelsize=20)


axs[0].set_ylabel('runtime (s)', fontsize=FONTSIZE_Y)
axs[0].set_xlabel('dimension', fontsize=FONTSIZE_X)
axs[1].set_xlabel('dimension', fontsize=FONTSIZE_X)
axs[1].set_ylabel('MSE', fontsize=FONTSIZE_Y)
plt.tight_layout(rect=[0, 0, 1, 0.90])
fig.legend(fontsize=30, loc='upper center', bbox_to_anchor=(0.52, 1.01), ncol=3)
#plt.show()
plt.savefig("figures/runtime_vs_Tyler.pdf")


         
         






NameError: name 'np' is not defined